In [34]:
import unidecode
from nltk.translate.bleu_score import sentence_bleu
import random as rd

play_file_infixes = ['othello', 'antony-and-cleopatra', 'asyoulikeit', 
                     'errors', 'hamlet', 'henryv', 'juliuscaesar', 'lear', 'macbeth', 
                     'merchant', 'msnd', 'muchado', 'richardiii', 'romeojuliet', 
                     'shrew', 'tempest', 'twelfthnight']
play_file_infixes = ['othello', 'henryv']

agg_original_tuples = []
agg_modern_tuples = []
path = "data/shakespeare/data/align/plays/merged/"

for infix in play_file_infixes:
    modern_tuples = unidecode.unidecode(open(path + infix + "_modern.snt.aligned").read()).split("\n")
    original_tuples = unidecode.unidecode(open(path + infix + "_original.snt.aligned").read()).split("\n")
    agg_original_tuples.extend(original_tuples)
    agg_modern_tuples.extend(modern_tuples)

In [35]:
print("{}\n{}\n".format(agg_original_tuples[0], agg_modern_tuples[0]))
#print("{}\n{}\n".format(agg_original_tuples[5000], agg_modern_tuples[5000]))
# print("{}\n{}\n".format(agg_original_tuples[9000], agg_modern_tuples[9000]))
# print("{}\n{}\n".format(agg_original_tuples[18000], agg_modern_tuples[18000]))


What's the matter, lieutenant?
What's the matter, lieutenant?



In [36]:
# in-class = original = '1'
# raw -> (raw text, label)
import numpy as np

style1_np = np.array(agg_original_tuples)
style1_labels = np.array(['ori' for x in range(len(agg_original_tuples))])
X = np.vstack((style1_np, style1_labels))

style2_np = np.array(agg_modern_tuples)
style2_labels = np.array(['mod' for x in range(len(agg_modern_tuples))])
X2 = np.vstack((style2_np, style2_labels))

X = np.hstack((X,X2))
raw = np.transpose(X)

# idx_col = [str(x) for x in range(len(raw))]
# idx_col = np.array(idx_col).reshape(-1,1)
# print(idx_col.shape)
print(raw.shape)
#raw = np.hstack((idx_col, raw))
raw.shape
raw = np.random.permutation(raw)

(5844, 2)


In [37]:
raw[:10]

array([['No, great king.', 'ori'],
       ['What, are you hurt, lieutenant?', 'ori'],
       ['We are fortunate in the change.', 'mod'],
       ["But first, I have to tell you that Desdemona's completely in love with him.",
        'mod'],
       ['He speaks bluntly, madam.', 'mod'],
       ['Tis destiny unshunnable, like death.', 'ori'],
       ['Wait for mine, sweet soul.', 'mod'],
       ['When he is gone I would on great occasion speak with you.',
        'ori'],
       ['That she was false to wedlock?', 'ori'],
       ['Yes, unfortunately.', 'mod']], dtype='<U549')

In [68]:
from sklearn.feature_extraction import DictVectorizer
import spacy
import scipy.sparse
nlp = spacy.load('en_core_web_sm')

vectorizer = DictVectorizer()
features = []

# feature engineering
    # construct index column
# idx_col = np.array([raw[:,0]])
# idx_col = idx_col.astype('f', copy=True)

#bleu: 0-1, 1 is perfect score
#bleu = sentence_bleu([sen2],sen1)

raw_text_col = 0
LABEL_COL = -1

def extract_feats(row_idx, raw):
    feature_dict = {}
    punc_dict = {'colon_count': 0, 'semicolon_count':0, 'lparen_count': 0,
                 'ellipse_count': 0, 'quote_count': 0}
    pos_dict = {'NOUN_count': 0, 'ADV_count': 0, 'VERB_count': 0, 
                'ADJ_count': 0, 'adv_verb_ratio': 0, 'adj_noun_ratio': 0
               }
    txt = str(raw[row_idx][raw_text_col])
    
    feature_dict['sent_len'] = len(txt)
    try:
        doc = nlp(txt)
    except IndexError as e:
        print(e)
        print("offending message [{}], len: {}, row idx: {}".format(txt, len(txt), row_idx))

    for token in doc:
        pos = token.pos_
        pos_dict[pos+'_count'] = pos_dict.get(pos+'_count', 0) + 1

    #      import re
    #     p = re.compile('\'')
    #     p.findall("' ' '")

    if pos_dict['NOUN_count'] == 0 or pos_dict['ADJ_count'] == 0:
        pos_dict['adj_noun_ratio'] = 0
    else:
        pos_dict['adj_noun_ratio'] = pos_dict['NOUN_count'] / pos_dict['ADJ_count'] 

    if pos_dict['VERB_count'] == 0 or pos_dict['ADV_count'] == 0:
        pos_dict['adv_verb_ratio'] = 0
    else:
        pos_dict['adv_verb_ratio'] = pos_dict['VERB_count'] / pos_dict['ADV_count'] 
        
    feature_dict.update(pos_dict)
    feature_dict.update(punc_dict)
    
    if row_idx % 1000 == 0:
        print("{} gold: {} {}\n".format(txt, raw[row_idx][LABEL_COL], feature_dict) )
     
    return feature_dict


for row_idx,_ in enumerate(raw):
    features.append(extract_feats(row_idx, raw))

#labels_col = np.array([raw[:,LABEL_COL]])

X = vectorizer.fit_transform(features)
print(X.shape)

#X = scipy.sparse.hstack(( idx_col.T, X, labels_col.T ))
X = X.todense()


No, great king. gold: ori {'sent_len': 15, 'NOUN_count': 1, 'ADV_count': 0, 'VERB_count': 0, 'ADJ_count': 1, 'adv_verb_ratio': 0, 'adj_noun_ratio': 1.0, 'INTJ_count': 1, 'PUNCT_count': 2, 'colon_count': 0, 'semicolon_count': 0, 'lparen_count': 0, 'ellipse_count': 0, 'quote_count': 0}

I will hear further reason for this. gold: ori {'sent_len': 36, 'NOUN_count': 1, 'ADV_count': 0, 'VERB_count': 2, 'ADJ_count': 1, 'adv_verb_ratio': 0, 'adj_noun_ratio': 1.0, 'PRON_count': 1, 'ADP_count': 1, 'DET_count': 1, 'PUNCT_count': 1, 'colon_count': 0, 'semicolon_count': 0, 'lparen_count': 0, 'ellipse_count': 0, 'quote_count': 0}

Good love, call him back. gold: ori {'sent_len': 25, 'NOUN_count': 1, 'ADV_count': 1, 'VERB_count': 1, 'ADJ_count': 1, 'adv_verb_ratio': 1.0, 'adj_noun_ratio': 1.0, 'PUNCT_count': 2, 'PRON_count': 1, 'colon_count': 0, 'semicolon_count': 0, 'lparen_count': 0, 'ellipse_count': 0, 'quote_count': 0}

Go, and bring them. gold: ori {'sent_len': 19, 'NOUN_count': 0, 'ADV_count': 

In [69]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm

import time
import math
###############
# training ####
###############
# test breakout: 3500, 210
train_val_split_delim = math.floor(len(X) * .8)
val_test_split_delim = math.floor(len(X) * .9)
print("total rows: {} train val split delim: {} val test split delim: {}".format(len(X), train_val_split_delim, val_test_split_delim))
# numpy note: addressing [:delim] is exclusive of delim, but
#                         [delim:] is inclusive of delim


X_train = X[0:train_val_split_delim]
Y_train = raw[0:train_val_split_delim,LABEL_COL]

X_val = X[train_val_split_delim:val_test_split_delim]
Y_val = raw[train_val_split_delim:val_test_split_delim,LABEL_COL]

X_test = X[val_test_split_delim:]
Y_test = raw[val_test_split_delim:,LABEL_COL]

if len(X_test) != len(Y_test):
    raise ValueError("mismatched arrays! {} {}".format(len(preds), len(y_true)))


#mdl = LogisticRegression(random_state=123)
#mdl = MultinomialNB()
mdl = svm.SVC(kernel = 'linear', C=.1) # Accuracy: 0.589743
mdl.fit(X_train, Y_train)

np.set_printoptions(suppress=True)


total rows: 5844 train val split delim: 4675 val test split delim: 5259


In [70]:
# evaluation ###
################
#second class is original

from sklearn.metrics import brier_score_loss

#y_true = raw[train_val_split_delim:val_test_split_delim,LABEL_COL]
preds = mdl.predict(X_test)
#y_true = [int(raw[int(idx)][-1]) for idx in val_indices]

# brier = brier_score_loss(y_true, preds[:,0]) 
# print("brier score:", brier)

#currently slightly worse than randomly permuting preds

def accuracy(preds, y_true):
    
    if len(preds) != len(y_true):
        raise ValueError("mismatched arrays! {} {}".format(len(preds), len(y_true)))
    num_correct = 0
    for (x,y) in zip(preds, y_true):
        if x == y:
            num_correct = num_correct + 1
    return num_correct/len(preds)

print("Accuracy: {}".format(accuracy(preds,Y_test)))
print("Predictions evaluated: {}".format(len(preds)))

Accuracy: 0.5897435897435898
Predictions evaluated: 585


In [ ]:
# error analysis ##
##################


from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import itertools

conf_mat = confusion_matrix(y_true, preds)
# fig, ax = plt.subplots(figsize=(10,10))
# # sns.heatmap(conf_mat, annot=True, fmt='d',
# #             xticklabels=category_id_df.Product.values, yticklabels=category_id_df.Product.values)
# plt.ylabel('Actual')
# plt.xlabel('Predicted')

# NOTE: this function taken from: 
# http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html#sphx-glr-auto-examples-model-selection-plot-confusion-matrix-py
def plot_confusion_matrix(cm, classes,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f'
    thresh = 80
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')


plt.figure()
print(conf_mat)
plot_confusion_matrix(conf_mat, classes=mdl.classes_,
                      title="Confusion matrix")
plt.show()

In [ ]:
# test_len = len(preds)
# accuracy = sum(preds)/float(test_len)
# print("accuracy: {}".format(accuracy))

#othello accuracy: 0.7428571428571429. but maybe not permuted
#all: accuracy: 0.328237942884228

#pos, no bleu, 2 plays
# Mult NB: .519

In [66]:
import re
p = re.compile('\'')
p.findall("' ' '")

["'", "'", "'"]